In [1]:
# see from https://aimatters.wordpress.com/2016/01/16/solving-xor-with-a-neural-network-in-tensorflow/

import tensorflow as tf
import time

XOR_X = [[0,0],[0,1],[1,0],[1,1]]
XOR_Y = [[0],[1],[1],[0]]

x_ = tf.placeholder(tf.float32, shape=[4,2], name = 'x-input')
y_ = tf.placeholder(tf.float32, shape=[4,1], name = 'y-input')

Weight1 = tf.Variable(tf.random_uniform([2,2], -1, 1, seed=80636), name = "Weight1")
Weight2 = tf.Variable(tf.random_uniform([2,1], -1, 1, seed=80636), name = "Weight2")

Bias1 = tf.Variable(tf.zeros([2]), name = "Bias1")
Bias2 = tf.Variable(tf.zeros([1]), name = "Bias2")

with tf.name_scope("layer2") as scope:
	A2 = tf.sigmoid(tf.matmul(x_, Weight1) + Bias1)

with tf.name_scope("layer3") as scope:
	Hypothesis = tf.sigmoid(tf.matmul(A2, Weight2) + Bias2)

with tf.name_scope("cost") as scope:
	cost = tf.reduce_mean(( (y_ * tf.log(Hypothesis)) + 
		((1 - y_) * tf.log(1.0 - Hypothesis)) ) * -1)

with tf.name_scope("train") as scope:
	train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cost)

init = tf.global_variables_initializer()
sess = tf.Session()

writer = tf.summary.FileWriter("./logs/xor_logs/xor_tf", sess.graph)

sess.run(init)

t_start = time.clock()
for i in range(100001):
	sess.run(train_step, feed_dict={x_: XOR_X, y_: XOR_Y})
	if i % 10000 == 0:
		print('Epoch ', i)
		print('Hypothesis ', sess.run(Hypothesis, feed_dict={x_: XOR_X, y_: XOR_Y}))
		print('cost ', sess.run(cost, feed_dict={x_: XOR_X, y_: XOR_Y}))

print('Weight1 ', sess.run(Weight1))
print('Bias1 ', sess.run(Bias1))
print('Weight2 ', sess.run(Weight2))
print('Bias2 ', sess.run(Bias2))

t_end = time.clock()
print('Elapsed time ', t_end - t_start)

('Epoch ', 0)
('Hypothesis ', array([[0.5930999 ],
       [0.5729397 ],
       [0.61108035],
       [0.59025407]], dtype=float32))
('cost ', 0.7102268)
('Epoch ', 10000)
('Hypothesis ', array([[0.5035934 ],
       [0.4826417 ],
       [0.51913285],
       [0.49556032]], dtype=float32))
('cost ', 0.6921857)
('Epoch ', 20000)
('Hypothesis ', array([[0.505562  ],
       [0.46940672],
       [0.5379392 ],
       [0.49013022]], dtype=float32))
('cost ', 0.68855715)
('Epoch ', 30000)
('Hypothesis ', array([[0.48958266],
       [0.42055997],
       [0.6364423 ],
       [0.46262786]], dtype=float32))
('cost ', 0.6529052)
('Epoch ', 40000)
('Hypothesis ', array([[0.41351685],
       [0.35970157],
       [0.8433527 ],
       [0.39646748]], dtype=float32))
('cost ', 0.55785435)
('Epoch ', 50000)
('Hypothesis ', array([[0.35672814],
       [0.41164786],
       [0.9114599 ],
       [0.35968634]], dtype=float32))
('cost ', 0.4668199)
('Epoch ', 60000)
('Hypothesis ', array([[0.16094044],
       [0.8

In [2]:
print sess.run(Hypothesis, feed_dict={x_: XOR_X, y_: XOR_Y})

[[0.02589168]
 [0.9787202 ]
 [0.98138416]
 [0.02234969]]


In [3]:
freeze_var_names = list(set(v.op.name for v in tf.global_variables()))
print freeze_var_names
output_names = [Hypothesis.op.name]
print output_names

from tensorflow.python.framework.graph_util import remove_training_nodes

sub_graph_def = remove_training_nodes(sess.graph_def)


from tensorflow.python.framework import graph_util

frozen_graph = graph_util.convert_variables_to_constants(sess, 
                                                         sub_graph_def, 
                                                         output_names, 
                                                         freeze_var_names)

graph_path = tf.train.write_graph(frozen_graph, "models", "xor_tf.pb", as_text=False)
print('%s written' % graph_path)

[u'Bias1', u'Bias2', u'Weight2', u'Weight1']
[u'layer3/Sigmoid']
INFO:tensorflow:Froze 4 variables.
INFO:tensorflow:Converted 4 variables to const ops.
models/xor_tf.pb written


In [4]:
# utensor-cli convert models/xor2n.pb --output-nodes=layer3_3/Sigmoid
# unsupported op type in uTensor: Sigmoid